# Sprint8
## アンサンブル学習

**主要ライブラリインポート**

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools as itr
import random

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error


**データセット用意**

In [2]:
data_set=pd.read_csv("/Users/wakaichiaki/diveintocode-ml/diveintocode-ml/Week03/house-prices-advanced-regression-techniques/train.csv")

In [3]:
#説明変数として、GrLivAreaとYearBuilt,目的変数としてSalePrice
X,y=data_set[["GrLivArea","YearBuilt"]],data_set[["SalePrice"]]

#data_split(8:2)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

#標準化データ
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)

/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


## 【問題1】ブレンディングのスクラッチ実装

**１つ目(重みをづけ)**

In [4]:
#モデル定義
Lr=LinearRegression()
Sr=SVR()
Dt=DecisionTreeRegressor()

In [5]:
#fit
Lr.fit(X_train,y_train)
Sr.fit(X_train,y_train)
Dt.fit(X_train,y_train)
#pred
y_Lr=Lr.predict(X_test)
y_Sr=Sr.predict(X_test).reshape(-1,1)
y_Dt=Dt.predict(X_test).reshape(-1,1)
blend_1 = (y_Lr + y_Sr + y_Dt) / 3

/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [6]:
#MSE
Lr_MSE_1=mean_squared_error(y_test,y_Lr)
Sr_MSE_1=mean_squared_error(y_test,y_Sr)
Dt_MSE_1=mean_squared_error(y_test,y_Dt)
blend_MSE_1=mean_squared_error(y_test,blend_1)
print(
    "MSE:\n",
    "LinearRegression:",Lr_MSE_1,"\n",
    "SVR:",Sr_MSE_1,"\n",
    "DecisionTreeRegressor",Dt_MSE_1,"\n",
    "blend:",blend_MSE_1
     )

MSE:
 LinearRegression: 2035284873.0553393 
 SVR: 6403704560.755616 
 DecisionTreeRegressor 2959090711.952435 
 blend: 2558785215.962858


各モデルによる教師データと予測データのMSEから、SVRの予測精度が低いことがわかる。
２つ目では、SVRの影響を弱めるため重みを小さくしてみる。

In [7]:
#MSE割合
all_MSE=Lr_MSE_1+Sr_MSE_1+Dt_MSE_1
weight=[Lr_MSE_1/all_MSE, Sr_MSE_1/all_MSE, Dt_MSE_1/all_MSE]
weight

[0.1785638324197821, 0.5618230858936223, 0.25961308168659564]

In [8]:
#重み定義
w_1=0.001 #SVR
w_2_3=0.4
w_2,w_3=(1-w_1)/2-w_2_3,(1-w_1)/2+w_2_3 #LogisticRegression,DecisionTreeRegressor
#Blending
blend_1w=(y_Lr*w_2+y_Sr*w_1+y_Dt*w_3)/3
print("重み\nLr:{}\nSr:{}\nDt:{}".format(w_1,w_2,w_3))

重み
Lr:0.001
Sr:0.09949999999999998
Dt:0.8995


In [9]:
#MSE
blend_MSE_1w=mean_squared_error(y_test,blend_1w)
print("MSE\n","blend:",blend_MSE_1w)

MSE
 blend: 18802548016.135845


**２つ目(パラメータを調整してみる)**

In [10]:
#モデル定義
Lr=LinearRegression(normalize = True)
Sr=SVR(gamma="scale", kernel="linear")
Dt=DecisionTreeRegressor(max_depth=10)

In [11]:
#fit
Lr.fit(X_train,y_train)
Sr.fit(X_train,y_train)
Dt.fit(X_train,y_train)
#pred
y_Sr_2=Lr.predict(X_test)
y_Lr_2=Sr.predict(X_test).reshape(-1,1)
y_Dt_2=Dt.predict(X_test).reshape(-1,1)
#Blending
blend_2=(y_Lr_2*w_1+y_Sr_2*w_2+y_Dt_2*w_3)/3

/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
#MSE
Lr_MSE_2=mean_squared_error(y_test,y_Lr_2)
Sr_MSE_2=mean_squared_error(y_test,y_Sr_2)
Dr_MSE_2=mean_squared_error(y_test,y_Dt_2)
blend_MSE_2=mean_squared_error(y_test,blend_2)
print(
    "MSE\n",
    "LinearRegression:",Lr_MSE_2,"\n",
    "SVR:",Sr_MSE_2,"\n",
    "DecisionTreeRegressor",Dr_MSE_2,"\n",
    "blend:",blend_MSE_2
     )

MSE
 LinearRegression: 2142630055.3327105 
 SVR: 2035284873.0553389 
 DecisionTreeRegressor 2298748786.1120825 
 blend: 18733596300.748272


**3つ目(データセットを標準化してみる)**

In [13]:
#標準化したデータでのモデル定義
Lr_sc=LinearRegression()
Sr_sc=SVR(gamma="scale", kernel="linear")
Dr_sc=DecisionTreeRegressor(max_depth=10)

In [14]:
#fit
Lr.fit(X_train_sc,y_train)
Sr.fit(X_train_sc,y_train)
Dt.fit(X_train_sc,y_train)
#pred
y_Lr_sc=Lr.predict(X_test)
y_Sr_sc=Sr.predict(X_test).reshape(-1,1)
y_Dt_sc=Dt.predict(X_test).reshape(-1,1)
#Blending
blend_3=(y_Lr_sc*w_1+y_Sr_sc*w_2+y_Dt_sc*w_3)/3

/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
#MSE
Lr_MSE_3=mean_squared_error(y_test,y_Lr_sc)
Sr_MSE_3=mean_squared_error(y_test,y_Sr_sc)
Dt_MSE_3=mean_squared_error(y_test,y_Dt_sc)
blend_MSE_3=mean_squared_error(y_test,blend_3)
print(
    "MSE\n",
    "SGDRegressor:",Lr_MSE_3,"\n",
    "LogisticRegression:",Sr_MSE_3,"\n",
    "DecisionTreeRegressor",Dt_MSE_3,"\n",
    "blend:",blend_MSE_3
     )

MSE
 SGDRegressor: 1.9878993520104776e+16 
 LogisticRegression: 5352808693124.996 
 DecisionTreeRegressor 6500396215.897261 
 blend: 4166578716.0769796


## 【問題2】バギングのスクラッチ実装

In [41]:
df = pd.read_csv("train.csv")
X = df.loc[: , ["GrLivArea" , "YearBuilt"]]
y = df.loc[: , "SalePrice"]
std = StandardScaler()
X_std = std.fit_transform(X)
X_std = pd.DataFrame(data = X_std)
X_train , X_test ,y_train ,  y_test = train_test_split(
             X , y , test_size = 0.2)

/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/wakaichiaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [21]:
X_t = np.random.choice(X_train.index , X_train.shape[0] , replace=True)

In [26]:
sme_list = []
def bagging(model , X , y , n = 10):
    for i in range(n):
        X_train , X_test ,y_train ,  y_test = train_test_split(
             X , y , test_size = 0.2)
        X_index = np.random.choice(X_train.index , X_train.shape[0] , replace=True)
        X_train_n = X_train.loc[X_index]
        model.fit(X_train_n , y_train)
        pred = model.predict(X_test)
        sme = mean_squared_error(y_test , pred)
        sme_list.append(sme)
#     print("スコア{}".format(model.score(X_test , y_test)))
    return (np.mean(sme_list[-1]))

In [27]:
#baging
bagging(Lr , X , y)

6285608637.674811

## 【問題3】スタッキングのスクラッチ実装

**ステージ0**

In [47]:
#クロスバリデーション
from sklearn.model_selection import KFold
kf = KFold(n_splits = 4)

In [50]:
svm = SVR()
tree = DecisionTreeRegressor()
lgb = lgb.LGBMClassifier()
forest = RandomForestClassifier(max_depth = 2, min_samples_split = 2, n_estimators = 40, random_state =3)

**学習フェーズ**

In [51]:
models = [forest , lgb , SVR]
def Stacking(models , X_train , X_test , y_train):
    preds = np.array([])
    preds_test = []
    va_idxes = []
    kf = KFold(n_splits = 3 , random_state = 5)

    #クロスバリデーションで学習、予測を行い、予測値とインデックスを保存
    for m, model in enumerate(models):
        for i , (tr_idx , va_idx) in enumerate(kf.split(X_train)):
            tr_X , va_X = X_train.iloc[tr_idx] , X_train.iloc[va_idx]
            tr_y , va_y = y_train.iloc[tr_idx] , y_train.iloc[va_idx]
            model.fit(tr_X , tr_y) #va_X , va_y)
            model.fit(va_X , va_y)
            pred = model.predict(va_X)
            preds = np.append(preds , pred)
            pred_test = model.predict(X_test)
            preds_test = np.append(preds_test , pred_test)
            va_idxes.append(va_idx)
    #バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    preds = preds.reshape(1168 , len(models) )
    preds_test = preds_test.reshape(292 , 9)
    va_idxes = np.concatenate(va_idxes)
    #preds = np.concatenate(preds , axis = 0)
    order = np.argsort(va_idxes)
#    pred_train = preds[order]
    
    #テストデータに対する予測値の平均をとる
    preds_test_1 = np.mean(preds_test[: , :3] , axis = 1).reshape(-1,1)
    preds_test_2 = np.mean(preds_test[: , 3:6] , axis = 1).reshape(-1,1)
    preds_test_3 = np.mean(preds_test[: , 6:] , axis = 1).reshape(-1,1)
    preds_test_sum = np.concatenate([preds_test_1 , preds_test_2 , preds_test_3] , axis=1)
#     pred_train = pred_train.reshape(1168  , len(models))
    return preds_test_sum , preds

In [ ]:
preds_test , preds =Stacking(models , X_train ,X_test,y_train)

In [38]:
def model(X_train , X_test , y_train , model):
    model.fit(preds , y_train)
    pred = model.predict(preds_test)
    sme = mean_squared_error(y_test , pred)
    return sme

In [39]:
model(preds , preds_test , y_train , lr)

5634119204.318373